In [ ]:
import re

def trim_ltm_stm(ltm_list, placeholder="[...]", max_length=3000, list_trim_threshold=30):
    """
    Trims long text lists in the LTM entries and replaces them with a placeholder, 
    while preserving some examples for context. List of lists are trimmed only if too long.

    Args:
        ltm_list (list): List of strings containing the LTM queries.
        placeholder (str): Placeholder text to replace long lists.
        max_length (int): Max length of entire entry before truncation.
        list_trim_threshold (int): Maximum number of characters a list can have before being trimmed.

    Returns:
        list: Trimmed LTM list.
    """
    trimmed_ltm = []

    for entry in ltm_list:
        # Function to selectively trim long lists but keep part of the data
        def trim_list(match):
            content = match.group(0)  # The entire matched list
            if len(content) > list_trim_threshold:
                # Preserve the first few elements of the list while trimming the rest
                partial_content = re.sub(r"(\[.*?, .*?)\s*,\s*.*?\]", rf"\1, {placeholder} ]", content, flags=re.DOTALL)
                return partial_content.replace(f"[{placeholder},", f"[{placeholder}")  # Cleanup format
            return content  # If the list is short, return as is

        # Function to handle nested lists (list of lists)
        def trim_nested_list(match):
            content = match.group(0)  # The entire matched nested list
            if len(content) > list_trim_threshold:
                # Preserve some nested elements while trimming the rest
                partial_content = re.sub(r"(\[\[.*?\], \[.*?\])\s*,\s*.*?\]", rf"\1, {placeholder} ]", content, flags=re.DOTALL)
                return partial_content.replace(f"[{placeholder},", f"[{placeholder}")  # Cleanup format
            return content  # If the nested list is short, return as is

        # Trim only long nested lists (list of lists)
        trimmed_entry = re.sub(r"\[\s*(?:\[[^\]]*\]\s*,?\s*)+\]", trim_nested_list, entry, flags=re.DOTALL)  

        # Trim only long lists inside brackets
        trimmed_entry = re.sub(r"\[.*?\]", trim_list, trimmed_entry, flags=re.DOTALL)  

        # If the whole entry is still too long, truncate the text itself
        if len(trimmed_entry) > max_length:
            trimmed_entry = trimmed_entry[:max_length] + " " + placeholder

        trimmed_ltm.append(trimmed_entry)

    return trimmed_ltm

# Example usage with your LTM list
ltm_list = [
    'Query: "assistant\n\nWhat is the perplexity of the gpt2 model for the following texts: [\'The sun is shining brightly in the clear blue sky\', \'The cat purrs contentedly on my lap\', \'The dog wags its tail with excitement\', \'The baby laughs at the silly clown\', \'The flowers bloom beautifully in the garden\']? \n Solved: Yes',
    'Query: "assistant\n\nWhat is the accuracy of the model given the predictions [1, 0, 1, 1, 1, 0, 0, 1, 0, 1] and the references [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]? \n Solved: No',
    'Query: "What is the BLEU score for the translations [["hello", "world"], ["test", "example"], ["test", "example"], ["test", "example"], ["data", "more data"]] against [["reference", "gold"], ["correct", "baseline"]]?"'
]

trimmed_ltm = trim_ltm_stm(ltm_list)
for entry in trimmed_ltm:
    print(entry)

Query: "assistant

What is the perplexity of the gpt2 model for the following texts: ['The sun is shining brightly in the clear blue sky', 'The cat purrs contentedly on my lap', [SAMPLE_TEXTS] ]? 
 Solved: Yes
Query: "assistant

What is the accuracy of the model given the predictions [1, 0, 1, 1, 1, 0, 0, 1, 0, 1] and the references [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]? 
 Solved: No
Query: "What is the BLEU score for the translations [["hello", "world"], ["test", "example"], [SAMPLE_TEXTS] ], ["test", "example"], ["data", "more data"]] against [["reference", "gold"], ["correct", "baseline"]]?"


In [28]:
import json

file_path = "saved_results/data_dict_20250221_184302.json"

# Read the entire JSON file as a single string
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

print(f"Total characters in file: {len(content)}")

# Define the problematic position
error_position = 540793
context_size = 500# Number of characters before and after the error

# Ensure we don't go out of bounds
start_index = max(0, error_position - context_size)
end_index = min(len(content), error_position + context_size)

# Print the suspect portion
print("===== Suspicious JSON Fragment =====")
print(content[start_index:end_index])
print("===================================")

Total characters in file: 546701
===== Suspicious JSON Fragment =====
or a list of *N* lists of references. Each reference should be a string with tokens separated by spaces.", "default": ""}], "optional_param fox jumps over the lazy dog\"],\n    \"rouge_types\": [\"rouge1\", \"rouge2\", \"rougeL\", \"rougeLsum\"],\n    \"use_aggregator\": true,\n    \"use_stemmer\": false\n}\n\nFinal Answer: The ROUGE metric provides a good evaluation of the given query, but it may be beneficial to consider other metrics to get a more comprehensive understanding of the results."}, {"role": "user", "content": "\"Do you think you successfully fulfilled this query in the end? Respond with \\\"Yes\\\" or \\\"No\\\".\""}, {"role": "assistant", "content": "\"assistant\n\nYes"}, {"role": "user", "content": "Now you know a bit more about the available evaluation metrics. You can synthesize another user query to explore the metrics further and consolidate your understanding based on what you discovered. Again,

In [ ]:
import json

# Load saved results
with open("saved_results/data_dict_20250221_184302.json", "r", encoding="utf-8") as f:
    data_dict = json.load(f)

# Print all stored APIs
print("APIs tested:", list(data_dict.keys()))

# View first session of a specific API
api_name = "accuracy"
print("First session for API:", api_name)
print(json.dumps(data_dict[api_name][0], indent=4))

JSONDecodeError: Expecting ':' delimiter: line 1 column 540793 (char 540792)

In [14]:
import json
js = '{"predictions": [0, 1, 1, 0, 0, 1, 0, 1, 1, 0], "references": [0, 1, 0, 1, 1, 0, 1, 0, 0, 1], "normalize": false}'
json.loads(str(js).strip())

{'predictions': [0, 1, 1, 0, 0, 1, 0, 1, 1, 0],
 'references': [0, 1, 0, 1, 1, 0, 1, 0, 0, 1],
 'normalize': False}

In [ ]:
import evaluate
accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
print(results)

{'accuracy': 1.0}


In [6]:
import evaluate

#"predictions": ["The sun is shining brightly in the sky", "The cat is sleeping on the couch", "The dog is running in the park"]
#"references": [["The sun is shining in the sky", "The cat is sleeping on the couch", "The dog is running in the park"], ["The sun is shining in the clear blue sky", "The cat is sleeping on the soft couch", "The dog is running around the park"]]
#"metric_name": "bleu"
predictions = ["The sun is shining brightly in the sky", "The cat is sleeping on the couch", "The dog is running in the park"]
references = [["The sun is shining in the sky", "The cat is sleeping on the couch", "The dog is running in the park"], ["The sun is shining in the clear blue sky", "The cat is sleeping on the soft couch", "The dog is running around the park"], ["The sun is shining in the clear blue sky", "The cat is sleeping on the soft couch", "The dog is running around the park"]]
metrics = [  
  "bleu",
  "bertscore",
  "perplexity",
  "rouge",
  "accuracy",
  "exact_match",
  "recall",
  "f1"
]
for metric_name in metrics:
    try :
        args = {
            "predictions": predictions,
            "references": references
        }
        metric = evaluate.load(metric_name)
        result = metric.compute(**args)
        print(result)
    except Exception as e:
        print("Error: ", e)
        pass

{'bleu': 0.6354346056137057, 'precisions': [0.9545454545454546, 0.7894736842105263, 0.5625, 0.38461538461538464], 'brevity_penalty': 1.0, 'length_ratio': 1.0476190476190477, 'translation_length': 22, 'reference_length': 21}
Error:  Either 'lang' (e.g. 'en') or 'model_type' (e.g. 'microsoft/deberta-xlarge-mnli') must be specified
Error:  Perplexity._compute() missing 1 required positional argument: 'model_id'
{'rouge1': 0.9079365079365079, 'rouge2': 0.735042735042735, 'rougeL': 0.9079365079365079, 'rougeLsum': 0.9079365079365079}
Error:  invalid literal for int() with base 10: 'The sun is shining brightly in the sky'
Error:  Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ['The sun is shining brightly in the sky', 'The cat is sleeping on the couch', 'The dog is running in the park'],
Input references: [['The sun is shining in the s

In [2]:
    response_text = "blablabla <|promptends|> bububu"
    parts = response_text.split("<|promptends|>")
    if parts:
        response_text = parts[1].strip()
        print(response_text)
    else:
        print("DEBUG ERROR: <|promptends|> not found")

bububu


In [3]:
print(len(" [SYSTEM]: You are a helpful assistant.\
[USER]: You are an **expert** assisting in evaluation.   \
Your task is to **output a single, well-structured query** that a user might naturally ask when requesting a metric evaluation.  \
**Context:**\
API_name: exact_match\
Description: {'description': 'Exact Match computes the percentage of predictions that exactly match the reference answers, a common metric in question answering and similar tasks.', 'required_parameters': [{'name': 'predictions', 'type': 'LIST of STRING', 'description': 'TList of predicted texts.', 'default': ''}, {'name': 'references', 'type': 'LIST of STRING', 'description': 'List of reference texts.', 'default': ''}], 'optional_parameters': [{'name': 'regexes_to_ignore', 'type': 'LIST of STRING', 'description': 'Regex expressions of characters to ignore when calculating the exact matches.', 'default': 'None'}, {'name': 'ignore_case', 'type': 'BOOLEAN', 'description': 'If True, turns everything to lowercase so that capitalization differences are ignored.', 'default': 'False'}, {'name': 'ignore_numbers (bool)', 'type': 'BOOLEAN', 'description': 'If True, removes all digits before comparing strings', 'default': 'False'}, {'name': 'ignore_punctuation (bool)', 'type': 'BOOLEAN', 'description': 'If True, removes punctuation before comparing strings.', 'default': 'False'}], 'example': {'predictions': ['Paris', 'London', 'Berlin'], 'references': ['Paris', 'London', 'Rome']}}\
\
**Task Instructions:**  \
Generate **one** realistic user query.  \
The query should be **concise, natural, and human-like**.  \
The query should **only** request metric evaluation **for a set references and predictions**.  \
It should provide parameters.   \
The query should provide very creative, diverse and long references and predictions.   \
Do **not** add explanations, descriptions, or metadata.  \
Do **not** repeat yourself.  \
Do **not** format the query as JSON or a code block.  \
**Stop after outputting the query.**\
\
User Query:\
"))

print(len("[SYSTEM]: You are a helpful assistant.\n[USER]: You are an **expert** assisting in evaluation.   \nYour task is to **output a single, well-structured query** that a user might naturally ask when requesting a metric evaluation.  \n**Context:**\nAPI_name: exact_match\nDescription: {'description': 'Exact Match computes the percentage of predictions that exactly match the reference answers, a common metric in question answering and similar tasks.', 'required_parameters': [{'name': 'predictions', 'type': 'LIST of STRING', 'description': 'TList of predicted texts.', 'default': ''}, {'name': 'references', 'type': 'LIST of STRING', 'description': 'List of reference texts.', 'default': ''}], 'optional_parameters': [{'name': 'regexes_to_ignore', 'type': 'LIST of STRING', 'description': 'Regex expressions of characters to ignore when calculating the exact matches.', 'default': 'None'}, {'name': 'ignore_case', 'type': 'BOOLEAN', 'description': 'If True, turns everything to lowercase so that capitalization differences are ignored.', 'default': 'False'}, {'name': 'ignore_numbers (bool)', 'type': 'BOOLEAN', 'description': 'If True, removes all digits before comparing strings', 'default': 'False'}, {'name': 'ignore_punctuation (bool)', 'type': 'BOOLEAN', 'description': 'If True, removes punctuation before comparing strings.', 'default': 'False'}], 'example': {'predictions': ['Paris', 'London', 'Berlin'], 'references': ['Paris', 'London', 'Rome']}}\n\n**Task Instructions:**  \nGenerate **one** realistic user query.  \nThe query should be **concise, natural, and human-like**.  \nThe query should **only** request metric evaluation **for a set references and predictions**.  \nIt should provide parameters.   \nThe query should provide very creative, diverse and long references and predictions.   \nDo **not** add explanations, descriptions, or metadata.  \nDo **not** repeat yourself.  \nDo **not** format the query as JSON or a code block.  \n**Stop after outputting the query.**\n\nUser Query:\n"))

1970
1988


In [65]:
# Code to generate a random metric subgroup with optional flags for each metric.

import json
import random
import os

def get_random_metric_subgroup_with_flags(json_path="tool_metadata/API_subgroups.json"):
    """
    Loads the metric subgroups from a JSON file, randomly selects one subgroup,
    and for each metric in that subgroup, generates a boolean flag that is True
    with 30% probability (and False with 70% probability).

    Parameters:
        json_path (str): Path to the JSON file containing the metric subgroups.

    Returns:
        parameters: A parametersionary with the following keys:
            - "name": The name of the selected subgroup.
            - "metrics": A list of metric names in the subgroup.
            - "optional_flags": A parametersionary mapping each metric name to a boolean flag.
    """
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"Subgroups JSON file not found at: {json_path}")
    
    with open(json_path, "r", encoding="utf-8") as f:
        subgroups = json.load(f)
    
    # Randomly choose one subgroup from the parametersionary values.
    chosen_subgroup = random.choice(list(subgroups.values()))
    
    # For each metric in the subgroup, assign a boolean flag (True with probability 0.3).
    optional_flags = {metric: (random.random() < 0.3) for metric in chosen_subgroup["metrics"]}
    
    return {
        "name": chosen_subgroup["name"],
        "metrics": chosen_subgroup["metrics"],
        "optional_flags": optional_flags
    }

# Example usage:
#if __name__ == "__main__":
subgroup_with_flags = get_random_metric_subgroup_with_flags()
print("Selected subgroup:", subgroup_with_flags["name"])
print("Metrics in subgroup:", subgroup_with_flags["metrics"])
print("Optional parameter flags:")
for metric, flag in subgroup_with_flags["optional_flags"].items():
    print(f"  {metric}: {flag}")

Selected subgroup: classification_exact
Metrics in subgroup: ['exact_match', 'recall', 'f1']
Optional parameter flags:
  exact_match: False
  recall: False
  f1: False


In [1]:
!pip install evaluate nltk absl-py rouge-score sacrebleu torch transformers numpy scipy scikit-learn bert_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=724072b1a4b5855afb55be76c4004fd54fc018c77e5bf782d26acb06ddf05e05
  Stored in directory: /Users/a39328/Library/Caches/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [14]:
import evaluate
import json
import os

# Load the API metrics from the JSON file.
api_metrics_json_path = "tool_metadata/API_list.json"
if not os.path.exists(api_metrics_json_path):
    raise FileNotFoundError(f"API metrics JSON file not found at: {api_metrics_json_path}")

with open(api_metrics_json_path, "r", encoding="utf-8") as f:
    api_metrics = json.load(f)

# Dictionary specifying the input format for each metric
metric_inputs = {
    "rouge": {"predictions": ["Once upon a time, there was a brave hero."], 
              "references": ["Once upon a time, a fearless warrior embarked on a journey."]},
    "bleu": {"predictions": ["Once upon a time, there was a brave hero."], 
             "references": [["Once upon a time, a fearless warrior embarked on a journey."]]},
    "bertscore": {"predictions": ["Once upon a time, there was a brave hero."], 
                  "references": ["Once upon a time, a fearless warrior embarked on a journey."], "lang": "en"},
    "perplexity": {"predictions": ["Once upon a time, there was a brave hero."], "model_id": "gpt2"},
    "accuracy": {"predictions": [1, 0, 1, 1], "references": [1, 0, 1, 0]},
    "exact_match": {"predictions": ["Once upon a time, there was a brave hero."], 
                    "references": ["Once upon a time, there was a brave hero."]},
    "recall": {
        "predictions": [1, 0, 1, 1], 
        "references": [1, 0, 1, 0]  # Changed to numeric classification labels
    },
    "f1": {
        "predictions": [1, 0, 1, 1], 
        "references": [1, 0, 1, 0]  # Changed to numeric classification labels
    }
}

# Evaluate each metric in the API list.
for metric_name in api_metrics:
    if metric_name not in metric_inputs:
        print(f"Skipping {metric_name}: No input data provided.")
        continue
    
    print(f"Evaluating metric: {metric_name}")

    try:
        # Load the metric
        metric = evaluate.load(metric_name)

        # Compute the metric
        results = metric.compute(**metric_inputs[metric_name])

        # Print results
        print(f"Results for {metric_name}: {results}")
    except Exception as e:
        print(f"Error evaluating {metric_name}: {e}")

Evaluating metric: rouge
Results for rouge: {'rouge1': 0.5, 'rouge2': 0.33333333333333326, 'rougeL': 0.5, 'rougeLsum': 0.5}
Evaluating metric: bleu
Results for bleu: {'bleu': 0.3181877033696365, 'precisions': [0.6363636363636364, 0.4, 0.3333333333333333, 0.25], 'brevity_penalty': 0.8337529180751805, 'length_ratio': 0.8461538461538461, 'translation_length': 11, 'reference_length': 13}
Evaluating metric: bertscore


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Results for bertscore: {'precision': [0.9666957259178162], 'recall': [0.9491630792617798], 'f1': [0.9578492045402527], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.48.1)'}
Evaluating metric: perplexity


  0%|          | 0/1 [00:00<?, ?it/s]

Results for perplexity: {'perplexities': [14.477904319763184], 'mean_perplexity': 14.477904319763184}
Evaluating metric: accuracy
Results for accuracy: {'accuracy': 0.75}
Evaluating metric: exact_match
Results for exact_match: {'exact_match': 1.0}
Evaluating metric: recall
Results for recall: {'recall': 1.0}
Evaluating metric: r_squared


Error evaluating r_squared: r_squared._compute() got an unexpected keyword argument 'squared'
Evaluating metric: f1
Results for f1: {'f1': 0.8}


In [ ]:
resp =''

In [ ]:
response = "this is a test"
print(len(response))
print(response[:len(response)])

14
this is a test


In [1]:
from evaluate import load

# Load the BLEU metric
bleu = load("bleu")
# Example predictions and references
predictions = ["The cat is sitting on the mat", "The dog is sleeping on the bed"]

references = [["The cat is sitting on the mat", "The dog is sleeping on the bed"]]

# Compute BLEU score
results = bleu.compute(predictions=predictions, references=references)

# Display results
print("BLEU Score:", results)

ValueError: Mismatch in the number of predictions (2) and references (1)